In [1]:
from datetime import datetime, date
import types
from typing import Callable, List, Optional, Union

import ibis
from ibis.expr import datatypes as dt
from ibis.expr import operations as ops
from ibis.expr import types as ir
from ibis.udf import vectorized

import numpy as np
import pandas as pd
import rbc
# local
from settings import conf
from utils import cursor2df
from gen_data import salary_date, salary_date_schema

In [2]:
con = ibis.omniscidb.connect(**conf['omniscidb'])

In [3]:
sql = '''
create table logs (
  logtime timestamp,
  operation text,
  session text,
  execution_time double,
  total_time double
)
'''
con.drop_table('logs', force=True)
con.con.execute(sql)

In [4]:
con.list_tables()

['diamonds',
 'batting',
 'awards_players',
 'functional_alltypes',
 'geo',
 'mytable',
 'logs']

In [5]:
logs = con.table('logs')

In [6]:
time_dimension = logs.logtime.truncate('m').name('time_minute')

measures = [
    ibis.ifelse(logs.operation == 'SELECT', 1, 0).count().name('num_select_queries'), 
    ibis.ifelse(logs.operation == 'DELETE', 1, 0).count().name('num_delete_queries'), 
    ibis.case().when(logs.operation == 'SELECT', logs.session).else_(dt.null).end().nunique().name('num_sessions'), 
    logs.execution_time.mean().name('avg_execution_time'), 
    logs.total_time.mean().name('avg_total_time'),
]
filters = [logs.operation == 'SELECT']
metrics_by_time = logs.filter(filters).group_by([time_dimension]).aggregate(measures)

In [7]:
print(metrics_by_time.compile())

> /home/xmn/dev/quansight/ibis-project/ibis/ibis/omniscidb/operations.py(689)__init__()
-> self.translator = translator


(Pdb)  p op


*** NameError: name 'op' is not defined


(Pdb)  p op


*** NameError: name 'op' is not defined


...

> /home/xmn/dev/quansight/ibis-project/ibis/ibis/omniscidb/operations.py(689)__init__()
-> self.translator = translator


(Pdb)  c


> /home/xmn/dev/quansight/ibis-project/ibis/ibis/omniscidb/operations.py(689)__init__()
-> self.translator = translator


(Pdb)  c


SELECT DATE_TRUNC(MINUTE, "logtime") AS time_minute,
       count(CASE WHEN "operation" = 'SELECT' THEN 1 ELSE 0 END) AS num_select_queries,
       count(CASE WHEN "operation" = 'DELETE' THEN 1 ELSE 0 END) AS num_delete_queries,
       count(DISTINCT CASE WHEN "operation" = 'SELECT' THEN "session" ELSE NULL END) AS num_sessions,
       avg("execution_time") AS avg_execution_time,
       avg("total_time") AS avg_total_time
FROM logs
WHERE "operation" = 'SELECT'
GROUP BY time_minute


In [8]:
metrics_by_time.execute()

> /home/xmn/dev/quansight/ibis-project/ibis/ibis/omniscidb/operations.py(689)__init__()
-> self.translator = translator


(Pdb)  l


684  	class CaseFormatter:
685  	    """Case Formatter class."""
686  	
687  	    def __init__(self, translator, base, cases, results, default):
688  	        import pdb; pdb.set_trace()
689  ->	        self.translator = translator
690  	        self.base = base
691  	        self.cases = cases
692  	        self.results = results
693  	        self.default = default
694  	


(Pdb)  p base


None


...

> /home/xmn/dev/quansight/ibis-project/ibis/ibis/omniscidb/operations.py(689)__init__()
-> self.translator = translator


(Pdb)  c


> /home/xmn/dev/quansight/ibis-project/ibis/ibis/omniscidb/operations.py(689)__init__()
-> self.translator = translator


(Pdb)  c


,time_minute,num_select_queries,num_delete_queries,num_sessions,avg_execution_time,avg_total_time


In [9]:
t = con.table('functional_alltypes')

expr = ibis.case().when(t.string_col == '1', 'one').end().name('num_sessions')
print(expr.compile())

> /home/xmn/dev/quansight/ibis-project/ibis/ibis/omniscidb/operations.py(689)__init__()
-> self.translator = translator


(Pdb)  l


684  	class CaseFormatter:
685  	    """Case Formatter class."""
686  	
687  	    def __init__(self, translator, base, cases, results, default):
688  	        import pdb; pdb.set_trace()
689  ->	        self.translator = translator
690  	        self.base = base
691  	        self.cases = cases
692  	        self.results = results
693  	        self.default = default
694  	


(Pdb)  p base


None


...

Cast[string]
  Literal[null]
    None
  to:
    string


(Pdb)  p base


None


(Pdb)  q


BdbQuit: 

In [ ]:
expr.execute().head()